In [23]:
#导入库
from urllib.request import urlopen
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import requests
import re
import urllib.parse
import time
import random
import pdfkit
import subprocess
import pandas as pd
from tqdm import tqdm

In [9]:
headers_={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36 Edg/89.0.774.68'}
proxies = {
    'http': '127.0.0.1:7890',
    'https': '127.0.0.1:7890'
}
options = {
    'no-background': None,
    'page-size': 'Letter',
    # add other options as needed
}

In [10]:
def savePDF(url, output_path):
    name = url.replace(":", "_")
    name = name.replace("/", "_")
    name = name.replace(" ", "_")
    output_path = f'./{output_path}/{name}.pdf'

    # Define the wkhtmltopdf command with the --no-stop-slow-scripts option
    command = ['wkhtmltopdf', '--no-stop-slow-scripts', url, output_path]

    # Call the wkhtmltopdf command using subprocess
    result = subprocess.run(command, capture_output=True)

    # Check the result of the command
    if result.returncode != 0:
        print(f"下载失败，没有访问页面访问权限")
    else:
        print(f"PDF {url} saved to {output_path}")

In [30]:
class Queue(object):
    #初始化队列
    def __init__(self):
        self.items = []
        
    #入队
    def enqueue(self, item):
        self.items.append(item)
        
    #出队
    def dequeue(self):
        if self.is_Empty():
            print("当前队列为空！！")
        else:
            return self.items.pop(0)
        
    #判断是否为空
    def is_Empty(self):
        return self.items == []
        
    #队列长度
    def size(self):
        return len(self.items)
    
    #返回队头元素，如果队列为空的话，返回None
    def front(self):
        if self.is_Empty():
            print("当前队列为空！！")
        else:
            return self.items[len(self.items) - 1]


#获取页面中所有外链的列表
def getExterLinks(bs, exterurl):
    
    #找出所有以www或http开头且不包含当前URL的链接
    for link in bs.find_all('a', href = re.compile
                            ('^(http|www)((?!'+urlparse(exterurl).netloc+').)*$')):
        #按照标准，URL只允许一部分ASCII字符，其他字符（如汉字）是不符合标准的，
        #我们的链接网址可能存在汉字的情况，此时就要进行编码。
        link.attrs['href'] = urllib.parse.quote(link.attrs['href'],safe='?=&:/')
        if link.attrs['href'] is not None:
            if link.attrs['href'] not in externalLinks:
                queueExt.enqueue(link.attrs['href'])
                externalLinks.append(link.attrs['href'])
#                 print(link.attrs['href'])
#     return externalLinks

#获取页面中所以内链的列表    
def getInterLinks(bs, interurl):
    interurl = '{}://{}'.format(urlparse(interurl).scheme,
                                urlparse(interurl).netloc)
   
    #找出所有以“/”开头的内部链接
    for link in bs.find_all('a', href = re.compile
                            ('^(/|.*'+urlparse(interurl).netloc+')')):
        link.attrs['href'] = urllib.parse.quote(link.attrs['href'],safe='?=&:/')
        if link.attrs['href'] is not None:
            if link.attrs['href'] not in internalLinks:
        #startsWith()方法用来判断当前字符串是否是以另外一个给定的子字符串“开头”的
                if(link.attrs['href'].startswith('//')):
                    if interurl+link.attrs['href'] not in internalLinks:
                        queueInt.enqueue(interurl+link.attrs['href'])
                        internalLinks.append(interurl+link.attrs['href'])
                elif(link.attrs['href'].startswith('/')):
                    if interurl+link.attrs['href'] not in internalLinks:
                        queueInt.enqueue(interurl+link.attrs['href'])
                        internalLinks.append(interurl+link.attrs['href'])
                else:
                    queueInt.enqueue(link.attrs['href'])
                    internalLinks.append(link.attrs['href'])
#     return internalLinks

def deepLinks():
    num = queueInt.size()
    while num > 1:
        i = queueInt.dequeue()
        if i is None:
            break
        else:
#             print('访问的内链')
#             print("链接为：", i)
            if 'tencent://' in i and 'message' in i: 
                print("wrong url")
            else:
#                 try:
#                     savePDF(i, './csg')
#                 except Exception as e:
#                     # handle the exception
#                     print(f"无效的网页")
                
#                 print('找到的新外链')
        #         html = urlopen(i)
                html=requests.get(i, headers=headers_)
                time.sleep(random.random()*3)
                domain1 = '{}://{}'.format(urlparse(i).scheme, urlparse(i).netloc)
                bs = BeautifulSoup(html.content, 'html.parser')
                getExterLinks(bs, domain1)
                getInterLinks(bs, domain1)


def getAllLinks(url):
    global num
#     html = urlopen(url)
    headers_={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36 Edg/89.0.774.68'}
    html = requests.get(url,headers=headers_)
    time.sleep(random.random()*3) #模拟人类行为，间隔随机的时间
    domain = '{}://{}'.format(urlparse(url).scheme, urlparse(url).netloc)
    bs = BeautifulSoup(html.content, 'html.parser')

    getInterLinks(bs, domain)
    getExterLinks(bs, domain)
    deepLinks()

In [13]:
print(len(externalLinks))
print(len(internalLinks))
links = list(set(externalLinks + internalLinks))
print(len(links))

2855
410
3263


In [31]:
deep_links = []
for link in tqdm(links):
    try:
        queueInt = Queue()   #存储内链的队列
        queueExt = Queue()   #存储外链的队列

        externalLinks = []
        internalLinks = []
        getAllLinks(link)
        deep_links = list(set(externalLinks + internalLinks + deep_links))
    except Exception as e:
        # handle the exception
        print(f"无效的网页")

  0%|                                                                                         | 0/3263 [02:40<?, ?it/s]


KeyboardInterrupt: 

In [17]:
links 

['https://news.bjx.com.cn/html/20230801/1322683.shtml',
 'http://www.chinasmartgrid.com.cn/List-News?rid=121',
 'https://guangfu.bjx.com.cn/zt/2013SNEC/',
 'http://www.bjx.com.cn/zt/4_14/',
 'https://guangfu.bjx.com.cn/topic/2019gflh/',
 'https://news.bjx.com.cn/html/20230629/1316064.shtml',
 'https://news.bjx.com.cn/html/20230731/1322461.shtml',
 'https://market.bjx.com.cn/',
 'https://news.bjx.com.cn/html/20230712/1318897.shtml',
 'https://fd.bjx.com.cn/zhuanti/2016fdgflt/',
 'http://news.bjx.com.cn/html/20100825/254762.shtml',
 'http://www.bjx.com.cn//club.bjx.com.cn',
 'https://news.bjx.com.cn/html/20230703/1316440.shtml',
 'https://guangfu.bjx.com.cn/zt/2014manz03/',
 'https://news.bjx.com.cn/html/20220406/1215624.shtml',
 'https://news.bjx.com.cn/rankinglist/chuneng/',
 'https://b2b.bjx.com.cn/about/AD_Service.shtml',
 'http://news.bjx.com.cn/html/20111229/333449.shtml',
 'http://www.chinasmartgrid.com.cn/news/20230731/649437.shtml',
 'http://www.chinasmartgrid.com.cn/news/202303

In [7]:
for link in links:
    if 'tencent://' in link and 'message' in link: 
        print("wrong url")
    else:
        try:
            savePDF(link, 'NE电气')
        except Exception as e:
            # handle the exception
            print(f"无效的网页")

PDF http://www.csg.cn saved to ./csg/http___www.csg.cn.pdf
PDF http://ztjy.people.cn/n1/2023/0726/c457340-40043949.html saved to ./csg/http___ztjy.people.cn_n1_2023_0726_c457340-40043949.html.pdf
下载失败，没有访问页面访问权限
PDF http://www.powerchina.cn/ saved to ./csg/http___www.powerchina.cn_.pdf
下载失败，没有访问页面访问权限
PDF http://politics.people.com.cn/n1/2023/0509/c1024-32681415.html saved to ./csg/http___politics.people.com.cn_n1_2023_0509_c1024-32681415.html.pdf
PDF http://politics.people.com.cn/n1/2023/0509/c1024-32681416.html saved to ./csg/http___politics.people.com.cn_n1_2023_0509_c1024-32681416.html.pdf
PDF https://mp.weixin.qq.com/s/hXwUK2wQtLkCEC6lMwfVBA saved to ./csg/https___mp.weixin.qq.com_s_hXwUK2wQtLkCEC6lMwfVBA.pdf
PDF http://www.hainan.gov.cn saved to ./csg/http___www.hainan.gov.cn.pdf
PDF http://www.gxzf.gov.cn saved to ./csg/http___www.gxzf.gov.cn.pdf
PDF http://opinion.people.com.cn/n1/2023/0420/c1003-32668605.html saved to ./csg/http___opinion.people.com.cn_n1_2023_0420_c1003-32668

PDF http://politics.people.com.cn/n1/2023/0727/c1001-40044330.html saved to ./csg/http___politics.people.com.cn_n1_2023_0727_c1001-40044330.html.pdf
PDF https://mp.weixin.qq.com/s/C6r6oDThyMwAiXuLASm12w saved to ./csg/https___mp.weixin.qq.com_s_C6r6oDThyMwAiXuLASm12w.pdf
PDF http://www.sasac.gov.cn saved to ./csg/http___www.sasac.gov.cn.pdf
PDF http://politics.people.com.cn/n1/2023/0727/c1001-40044328.html saved to ./csg/http___politics.people.com.cn_n1_2023_0727_c1001-40044328.html.pdf
PDF https://mp.weixin.qq.com/s/13hwAzwy30XBxPa8OzJJ2g saved to ./csg/https___mp.weixin.qq.com_s_13hwAzwy30XBxPa8OzJJ2g.pdf
PDF http://www.csg.cn/index/gsmt/201512/t20151202_109176.html saved to ./csg/http___www.csg.cn_index_gsmt_201512_t20151202_109176.html.pdf
下载失败，没有访问页面访问权限
PDF https://mp.weixin.qq.com/s/7VC8yqnYdawU9vwzYvf1DA saved to ./csg/https___mp.weixin.qq.com_s_7VC8yqnYdawU9vwzYvf1DA.pdf
下载失败，没有访问页面访问权限
PDF http://95598.csg.cn/ saved to ./csg/http___95598.csg.cn_.pdf
PDF http://ztjy.people.cn/

In [8]:
# import subprocess

# url = 'https://www.bjx.com.cn//www.bjx.com.cn'
# output_path = './test.pdf'
 
# options = [
#     '--disable-javascript',
#     '--no-background',
#     # add other options as needed
# ]

# command = ['wkhtmltopdf'] + options + [url, output_path]
# subprocess.run(command, check=True)